In [1]:
import numpy as np

In [13]:
x = np.random.rand(2,1)
y = np.ones((1,1))

In [14]:
x, y

(array([[0.70123656],
        [0.285145  ]]),
 array([[1.]]))

In [5]:
# Activation Entity
class ReLU:

    def __init__(self):
        pass

    def forward(self, Z):
        return np.where(Z>0, Z, np.zeros_like(Z))

    def __call__(self, Z):
        return self.forward(Z)

    def grad(self, Z):
        return {"i": self.grad_i(Z)}

    def grad_i(self, Z):
        return np.diag(np.where(Z>0, np.ones_like(Z), np.zeros_like(Z)).reshape(-1))

class Sigmoid:
    def  __init__(self):
        pass

    def forward(self, Z):
        return 1/(1+np.exp(-Z))

    def __call__(self, Z):
        return self.forward(Z)

    def grad(self, Z):
        return {"i": self.grad_i(Z)}

    def grad_i(self, Z):
        y = self(Z)
        return np.diag((y*(1-y)).reshape(-1))

In [16]:
# Layer entity
class Dense:

    def __init__(self, no_of_neurons, input_size):

        self.W = np.random.randn(input_size, no_of_neurons)
        self.b = np.random.randn(no_of_neurons, 1)

    def __call__(self, X):
        return self.forward(X)

    def grad(self, X):
        return {"i": self.grad_i(X), "b": self.grad_b(X), "w": self.grad_w(X)}

    def grad_i(self, X):
        return self.W.T

    def grad_b(self, X):
        return np.identity(X.shape[0])

    def grad_w(self, X):
        m, n = self.W.shape
        return np.repeat(np.eye(n), repeats=m, axis=0)*np.repeat(np.expand_dims(X, 0), repeats=n, axis=0).reshape(m*n, 1)

    def forward(self, X):
        return np.matmul(self.W.T, X) + self.b

In [18]:
# Model entity
class Sequential:

    def __init__(self, layers=[]):
        self.layers = layers

    def __call__(self, X):
        return self.forward(X)

    def forward(self, X):
        for layer in self.layers:
            X = layer.forward(X)
            #gradient calculate
        return X

    def add(self, layer):
        self.layers.append(layer)
        return self

In [20]:
# Loss entity
class BinaryCrossEntropy:

    def __init__(self):
        pass

    def __call__(self, y_pred, y_true):
        return self.forward(y_pred, y_true)

    def forward(self, y_pred, y_true):
        return np.mean(np.where(y_true == 0, -np.log(1-y_pred), -np.log(y_pred)))

    # grad_i

In [21]:
layer1 = Dense(10, 2)
act1 = ReLU()
layer2 = Dense(10, 10)
act2 = ReLU()
layer3 = Dense(10, 10)
act3 = ReLU()
layer4 = Dense(1, 10)

In [22]:
model = Sequential(layers=[layer1, act1, layer2, act2, layer3, act3, layer4])

In [23]:
model.add(Sigmoid())

In [27]:
y_pred = model(x)
y_pred

array([[0.89872751]])

In [28]:
bce = BinaryCrossEntropy()
bce(y_pred, y)

0.10677539069210076